# First chart data processing

In [18]:
import pandas as pd
import numpy as np

In [19]:
csv = pd.read_csv('../data/GlobalSentiment_Data.csv', index_col = 0)

In [20]:
dictionary = {}

In [21]:
for index, countries in csv.groupby('iso3'):
    if countries.shape[0] is not 152:
        display(index)

In [22]:
csv[csv['iso3'] == 'TUR']

,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
19305,TUR,2020-01-01,1.734717,0.815279,NaN,2020-03-09,2020-03-25,NaN,NaN,NaN
19306,TUR,2020-01-02,0.556116,0.625424,NaN,2020-03-09,2020-03-25,NaN,NaN,NaN
19307,TUR,2020-01-03,0.506369,0.450007,NaN,2020-03-09,2020-03-25,NaN,NaN,NaN
19308,TUR,2020-01-04,0.400448,0.286395,NaN,2020-03-09,2020-03-25,NaN,NaN,NaN
19309,TUR,2020-01-05,0.246680,0.134107,NaN,2020-03-09,2020-03-25,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
19452,TUR,2020-05-27,0.013641,1.262619,-35.791809,2020-03-09,2020-03-25,68.0,2572.0,2070.0
19453,TUR,2020-05-28,0.928894,0.989178,-36.506757,2020-03-09,2020-03-25,60.0,3152.0,2364.0
19454,TUR,2020-05-29,1.722722,0.735273,-39.205882,2020-03-09,2020-03-25,56.0,3188.0,2282.0
19455,TUR,2020-05-30,0.468964,0.490497,-63.781879,2020-03-09,2020-03-25,52.0,2042.0,1966.0


In [24]:
for index, countries in csv.groupby('iso3'):
    countries.sort_values('date', inplace=True)
    dictionary[index] = {
        'senti_bert_std': list(countries['senti_bert_std']),
        'senti_bert_std_trend': list(countries['senti_bert_std_trend']),
        'transit': list(countries['transit']),
        'drop_senti_bert': list(countries['drop_senti_bert']),
        'min_senti_bert_std': list(countries['min_senti_bert_std']),
        'death': list(countries['death']),
        'recovered': list(countries['recovered']),
        'dates': list(countries['date']),
        'confirmed': list(countries['confirmed'])
    }

,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
1,AFG,2020-01-01,0.701896,0.028350,NaN,2020-03-10,2020-03-13,NaN,NaN,NaN
2,AFG,2020-01-02,0.823791,-0.083161,NaN,2020-03-10,2020-03-13,NaN,NaN,NaN
3,AFG,2020-01-03,-1.961022,-0.182154,NaN,2020-03-10,2020-03-13,NaN,NaN,NaN
4,AFG,2020-01-04,-0.505223,-0.274310,NaN,2020-03-10,2020-03-13,NaN,NaN,NaN
5,AFG,2020-01-05,-2.471051,-0.354487,NaN,2020-03-10,2020-03-13,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
148,AFG,2020-05-27,0.717064,1.251906,NaN,2020-03-10,2020-03-13,7.0,10.0,625.0
149,AFG,2020-05-28,0.346947,1.147359,NaN,2020-03-10,2020-03-13,8.0,71.0,580.0
150,AFG,2020-05-29,3.777787,1.041241,NaN,2020-03-10,2020-03-13,11.0,50.0,623.0
151,AFG,2020-05-30,1.241057,0.932738,NaN,2020-03-10,2020-03-13,3.0,44.0,866.0


,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
153,AGO,2020-01-01,1.748801,0.616943,NaN,2020-03-11,2020-03-23,NaN,NaN,NaN
154,AGO,2020-01-02,-0.948007,0.401770,NaN,2020-03-11,2020-03-23,NaN,NaN,NaN
155,AGO,2020-01-03,-0.679574,0.192538,NaN,2020-03-11,2020-03-23,NaN,NaN,NaN
156,AGO,2020-01-04,-1.767253,-0.016973,NaN,2020-03-11,2020-03-23,NaN,NaN,NaN
157,AGO,2020-01-05,1.628625,-0.225859,NaN,2020-03-11,2020-03-23,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
300,AGO,2020-05-27,1.293457,0.655108,-48.0,2020-03-11,2020-03-23,0.0,0.0,1.0
301,AGO,2020-05-28,-0.610434,0.556344,-41.5,2020-03-11,2020-03-23,0.0,0.0,3.0
302,AGO,2020-05-29,-1.147292,0.456079,-43.0,2020-03-11,2020-03-23,0.0,0.0,7.0
303,AGO,2020-05-30,1.159195,0.358416,-35.5,2020-03-11,2020-03-23,0.0,0.0,3.0


,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
305,ALB,2020-01-01,0.857488,-1.315886,NaN,2020-03-07,2020-03-25,NaN,NaN,NaN
306,ALB,2020-01-02,-4.032480,-1.343107,NaN,2020-03-07,2020-03-25,NaN,NaN,NaN
307,ALB,2020-01-03,-1.923233,-1.370036,NaN,2020-03-07,2020-03-25,NaN,NaN,NaN
308,ALB,2020-01-04,-1.161031,-1.397790,NaN,2020-03-07,2020-03-25,NaN,NaN,NaN
309,ALB,2020-01-05,-0.805836,-1.413446,NaN,2020-03-07,2020-03-25,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
452,ALB,2020-05-27,-0.419227,-0.109887,NaN,2020-03-07,2020-03-25,0.0,9.0,21.0
453,ALB,2020-05-28,-0.506397,-0.173215,NaN,2020-03-07,2020-03-25,0.0,11.0,26.0
454,ALB,2020-05-29,-0.332509,-0.237943,NaN,2020-03-07,2020-03-25,0.0,28.0,23.0
455,ALB,2020-05-30,0.057557,-0.302913,NaN,2020-03-07,2020-03-25,0.0,6.0,23.0


,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
457,ARE,2020-01-01,4.836081,2.236139,NaN,2020-03-07,2020-03-15,NaN,NaN,NaN
458,ARE,2020-01-02,1.720845,1.726068,NaN,2020-03-07,2020-03-15,NaN,NaN,NaN
459,ARE,2020-01-03,0.161223,1.222315,NaN,2020-03-07,2020-03-15,NaN,NaN,NaN
460,ARE,2020-01-04,-0.205698,0.718094,NaN,2020-03-07,2020-03-15,NaN,NaN,NaN
461,ARE,2020-01-05,-0.609683,0.221070,NaN,2020-03-07,2020-03-15,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
604,ARE,2020-05-27,-0.146897,0.452311,-50.714286,2020-03-07,2020-03-15,2.0,389.0,883.0
605,ARE,2020-05-28,-1.270816,-0.433615,-52.285714,2020-03-07,2020-03-15,3.0,314.0,563.0
606,ARE,2020-05-29,-0.427044,-1.320385,-58.000000,2020-03-07,2020-03-15,2.0,412.0,638.0
607,ARE,2020-05-30,-1.031135,-2.209622,-48.000000,2020-03-07,2020-03-15,2.0,449.0,726.0


,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
609,ARG,2020-01-01,9.537303,4.059912,NaN,2020-03-10,2020-03-16,NaN,NaN,NaN
610,ARG,2020-01-02,1.101754,3.210715,NaN,2020-03-10,2020-03-16,NaN,NaN,NaN
611,ARG,2020-01-03,-0.076065,2.400063,NaN,2020-03-10,2020-03-16,NaN,NaN,NaN
612,ARG,2020-01-04,0.383528,1.606204,NaN,2020-03-10,2020-03-16,NaN,NaN,NaN
613,ARG,2020-01-05,0.147128,0.826426,NaN,2020-03-10,2020-03-16,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
756,ARG,2020-05-27,0.875356,2.043228,-56.639423,2020-03-10,2020-03-16,16.0,182.0,705.0
757,ARG,2020-05-28,2.152221,2.069126,-57.164251,2020-03-10,2020-03-16,8.0,268.0,769.0
758,ARG,2020-05-29,2.546837,2.097571,-57.425926,2020-03-10,2020-03-16,12.0,171.0,717.0
759,ARG,2020-05-30,2.248039,2.131397,-61.275362,2020-03-10,2020-03-16,8.0,0.0,795.0


...

,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
20825,VNM,2020-01-01,1.448025,0.739775,NaN,2020-02-22,2020-03-19,NaN,NaN,NaN
20826,VNM,2020-01-02,0.995410,0.667537,NaN,2020-02-22,2020-03-19,NaN,NaN,NaN
20827,VNM,2020-01-03,-0.065745,0.596104,NaN,2020-02-22,2020-03-19,NaN,NaN,NaN
20828,VNM,2020-01-04,0.680420,0.521724,NaN,2020-02-22,2020-03-19,NaN,NaN,NaN
20829,VNM,2020-01-05,0.474809,0.444732,NaN,2020-02-22,2020-03-19,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
20972,VNM,2020-05-27,-0.287968,0.085106,-9.779661,2020-02-22,2020-03-19,NaN,NaN,NaN
20973,VNM,2020-05-28,-0.959118,0.245191,-8.271186,2020-02-22,2020-03-19,NaN,NaN,NaN
20974,VNM,2020-05-29,0.173521,0.412571,-3.431034,2020-02-22,2020-03-19,NaN,NaN,NaN
20975,VNM,2020-05-30,0.419224,0.591644,-0.627119,2020-02-22,2020-03-19,NaN,NaN,NaN


,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
20977,XKO,2020-01-01,1.419614,0.150087,NaN,2020-03-07,2020-03-12,NaN,NaN,NaN
20978,XKO,2020-01-02,-0.041168,0.033530,NaN,2020-03-07,2020-03-12,NaN,NaN,NaN
20979,XKO,2020-01-03,-1.027201,-0.071677,NaN,2020-03-07,2020-03-12,NaN,NaN,NaN
20980,XKO,2020-01-04,0.270538,-0.168653,NaN,2020-03-07,2020-03-12,NaN,NaN,NaN
20981,XKO,2020-01-05,-0.295190,-0.259909,NaN,2020-03-07,2020-03-12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
21124,XKO,2020-05-27,0.391642,0.565368,NaN,2020-03-07,2020-03-12,NaN,NaN,NaN
21125,XKO,2020-05-28,-0.565745,0.257490,NaN,2020-03-07,2020-03-12,NaN,NaN,NaN
21126,XKO,2020-05-29,-0.672479,-0.054952,NaN,2020-03-07,2020-03-12,NaN,NaN,NaN
21127,XKO,2020-05-30,2.045054,-0.369438,NaN,2020-03-07,2020-03-12,NaN,NaN,NaN


,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
21129,YEM,2020-01-01,-1.663804,-1.231925,NaN,2020-03-04,2020-03-18,NaN,NaN,NaN
21130,YEM,2020-01-02,0.498409,-1.012136,NaN,2020-03-04,2020-03-18,NaN,NaN,NaN
21131,YEM,2020-01-03,-2.551906,-0.792386,NaN,2020-03-04,2020-03-18,NaN,NaN,NaN
21132,YEM,2020-01-04,-0.586246,-0.579584,NaN,2020-03-04,2020-03-18,NaN,NaN,NaN
21133,YEM,2020-01-05,0.243314,-0.375423,NaN,2020-03-04,2020-03-18,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
21276,YEM,2020-05-27,1.384518,1.067670,-30.0,2020-03-04,2020-03-18,4.0,0.0,7.0
21277,YEM,2020-05-28,0.511617,0.994526,-33.0,2020-03-04,2020-03-18,4.0,1.0,22.0
21278,YEM,2020-05-29,0.638761,0.925068,-24.0,2020-03-04,2020-03-18,8.0,0.0,5.0
21279,YEM,2020-05-30,0.680368,0.859887,-20.0,2020-03-04,2020-03-18,12.0,2.0,27.0


,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
21281,ZAF,2020-01-01,1.570442,-0.312751,NaN,2020-03-08,2020-03-25,NaN,NaN,NaN
21282,ZAF,2020-01-02,-0.650670,-0.425268,NaN,2020-03-08,2020-03-25,NaN,NaN,NaN
21283,ZAF,2020-01-03,-1.475316,-0.524842,NaN,2020-03-08,2020-03-25,NaN,NaN,NaN
21284,ZAF,2020-01-04,-1.626520,-0.615249,NaN,2020-03-08,2020-03-25,NaN,NaN,NaN
21285,ZAF,2020-01-05,-1.071234,-0.699765,NaN,2020-03-08,2020-03-25,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
21428,ZAF,2020-05-27,-0.744527,0.808002,-52.7,2020-03-08,2020-03-25,28.0,710.0,1673.0
21429,ZAF,2020-05-28,-0.043204,0.736167,-52.0,2020-03-08,2020-03-25,25.0,919.0,1466.0
21430,ZAF,2020-05-29,2.114765,0.663140,-50.2,2020-03-08,2020-03-25,34.0,723.0,1837.0
21431,ZAF,2020-05-30,1.379471,0.588891,-43.0,2020-03-08,2020-03-25,32.0,1023.0,1727.0


,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
21433,ZMB,2020-01-01,2.299073,0.329223,NaN,2020-03-10,2020-03-25,NaN,NaN,NaN
21434,ZMB,2020-01-02,-0.166452,0.171433,NaN,2020-03-10,2020-03-25,NaN,NaN,NaN
21435,ZMB,2020-01-03,-1.397226,0.013945,NaN,2020-03-10,2020-03-25,NaN,NaN,NaN
21436,ZMB,2020-01-04,-0.292473,-0.152279,NaN,2020-03-10,2020-03-25,NaN,NaN,NaN
21437,ZMB,2020-01-05,-0.523435,-0.328566,NaN,2020-03-10,2020-03-25,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
21580,ZMB,2020-05-27,-1.000340,0.843584,-7.0,2020-03-10,2020-03-25,0.0,443.0,137.0
21581,ZMB,2020-05-28,-1.709906,0.503450,-4.5,2020-03-10,2020-03-25,0.0,0.0,0.0
21582,ZMB,2020-05-29,0.791908,0.142681,-8.0,2020-03-10,2020-03-25,0.0,0.0,0.0
21583,ZMB,2020-05-30,1.036297,-0.223535,-5.0,2020-03-10,2020-03-25,0.0,0.0,0.0


In [25]:
import json
json_object = json.dumps(dictionary, indent = 4)   


In [26]:
with open("../data/GlobalSentiment_Data.json", "w") as outfile:  
    json.dump(dictionary, outfile) 

# Map data processing

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

We need to show a map with the following variables: shock_standard and HalfLife from the dataset provided.

First, we need to load the two files and merge them to have a geojson with countries' geometries and with the aditional information.

In [2]:
provided_data = pd.read_csv("../data/Final_Output.csv")
provided_data.head()

,iso3,country,continent,min_senti_bert,drop_senti_bert,averageN,HalfLife,shock,shock_standard
0,AFG,Afghanistan,Asia,3/13/2020,3/10/2020,448.256579,8.410348,NaN,NaN
1,AGO,Angola,Africa,3/23/2020,3/11/2020,817.013158,12.320138,-2.156,-0.660154
2,ALB,Albania,Europe,3/25/2020,3/7/2020,288.138158,2.666999,NaN,NaN
3,ARE,United Arab Emirates,Asia,3/15/2020,3/7/2020,17878.046050,48.739074,-0.849,-0.686110
4,ARG,Argentina,South America,3/16/2020,3/10/2020,104898.302600,27.957062,-2.045,-0.879744


In [4]:
countries_shapes = gpd.read_file("../data/countries.geojson")
countries_shapes.head()

,ADMIN,ISO_A3,ISO_A2,geometry
0,Aruba,ABW,AW,"MULTIPOLYGON (((-69.99694 12.57758, -69.93639 ..."
1,Afghanistan,AFG,AF,"MULTIPOLYGON (((71.04980 38.40866, 71.05714 38..."
2,Angola,AGO,AO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,AIA,AI,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,ALB,AL,"MULTIPOLYGON (((19.74777 42.57890, 19.74601 42..."


Now, merge with ISO3 field.

In [5]:
countries_shapes = countries_shapes[countries_shapes['ISO_A3'] != '-99']

In [6]:
provided_data.drop([45], inplace=True)

In [7]:
provided_data[provided_data['iso3'] == 'GEO']

,iso3,country,continent,min_senti_bert,drop_senti_bert,averageN,HalfLife,shock,shock_standard
44,GEO,Georgia,Europe,3/13/2020,3/9/2020,333.611842,29.691186,NaN,NaN


In [11]:
merged_data = provided_data.merge(countries_shapes, right_on="ISO_A3", left_on="iso3", validate="1:1")
merged_data.head()

,iso3,country,continent,min_senti_bert,drop_senti_bert,averageN,HalfLife,shock,shock_standard,ADMIN,ISO_A3,ISO_A2,geometry
0,AFG,Afghanistan,Asia,3/13/2020,3/10/2020,448.256579,8.410348,NaN,NaN,Afghanistan,AFG,AF,"MULTIPOLYGON (((71.04980 38.40866, 71.05714 38..."
1,AGO,Angola,Africa,3/23/2020,3/11/2020,817.013158,12.320138,-2.156,-0.660154,Angola,AGO,AO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
2,ALB,Albania,Europe,3/25/2020,3/7/2020,288.138158,2.666999,NaN,NaN,Albania,ALB,AL,"MULTIPOLYGON (((19.74777 42.57890, 19.74601 42..."
3,ARE,United Arab Emirates,Asia,3/15/2020,3/7/2020,17878.046050,48.739074,-0.849,-0.686110,United Arab Emirates,ARE,AE,"MULTIPOLYGON (((53.86305 24.23469, 53.88860 24..."
4,ARG,Argentina,South America,3/16/2020,3/10/2020,104898.302600,27.957062,-2.045,-0.879744,Argentina,ARG,AR,"MULTIPOLYGON (((-68.65412 -54.88624, -68.65414..."


In [12]:
merged_data.keys()

Index(['iso3', 'country', 'continent', 'min_senti_bert', 'drop_senti_bert',
       'averageN', 'HalfLife', 'shock', 'shock_standard', 'ADMIN', 'ISO_A3',
       'ISO_A2', 'geometry'],
      dtype='object')

Filter in case we need full variables

In [13]:
merged_data = merged_data[['iso3', 'country', 'continent', 'min_senti_bert', 'drop_senti_bert',
       'averageN', 'HalfLife', 'shock', 'shock_standard', 'geometry', 'ISO_A2']]

Filter to get only the variables that we have discussed over slack

In [14]:
merged_data = merged_data[['iso3', 'country', 'continent', 'HalfLife', 'shock_standard', 'geometry', 'ISO_A2']]

In [15]:
merged_data = gpd.GeoDataFrame(merged_data)

In [16]:
merged_data.to_file("../data/countries_with_data.geojson", driver="GeoJSON")

In [17]:
merged_data['shock_standard'].describe(percentiles = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])

count    117.000000
mean      -0.848760
std        0.584025
min       -3.307960
10%       -1.610268
20%       -1.212389
30%       -1.000052
40%       -0.868296
50%       -0.723554
60%       -0.621621
70%       -0.485392
80%       -0.388376
90%       -0.230394
100%      -0.007038
max       -0.007038
Name: shock_standard, dtype: float64